Inspired by:
https://gist.github.com/ckiefer0/9b742305db2c7d2d47b8af776a6047f4

In [1]:
import os
import mne
import copy
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import zoom
from nilearn.datasets import fetch_atlas_aal
from nilearn.datasets import fetch_atlas_talairach

/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


# Setup
---

In [2]:
data_path = mne.datasets.sample.data_path()

Using default location ~/mne_data for sample...


  0%|          | Downloading : 0.00/1.54G [00:00<?,        ?B/s]

Error while fetching file https://osf.io/86qa2/download?version=5. Dataset fetching aborted.


OSError: [Errno 122] Disk quota exceeded

## AAL (Anatomical Automatic Labelling)

In [ ]:
aal_atlas = fetch_atlas_aal(data_dir = './Test/')

In [ ]:
aal_img = nib.load('./Test/aal_SPM12/aal/atlas/AAL.nii')

aal_labels = {}
for idx, label in enumerate(aal_atlas['labels']):
    aal_labels[label] = int(aal_atlas['indices'][idx])

aal_labels['0'] = 'Background'

aal_data = aal_img.get_fdata()

In [ ]:
plt.imshow(aal_data[:, :, 40])

## Talairach

In [ ]:
tal_atlas = fetch_atlas_talairach(level_name='ba', data_dir=data_path)

In [ ]:
tal_img = tal_atlas['maps']

tal_labels = {}
for idx, label in enumerate(tal_atlas['labels']):
    tal_labels[label] = idx
    
tal_data = tal_img.get_fdata()

In [ ]:
plt.imshow(tal_data[:, :, 40])

## PET scan

In [ ]:
pet_scan = nib.load('./Git/Master-thesis/Development/data/SPM_categorised/ADNI1/CN/' + os.listdir('./Git/Master-thesis/Development/data/SPM_categorised/ADNI1/CN/')[0]).get_fdata()
pet_scan_resized = zoom(pet_scan, (aal_data.shape[0] / pet_scan.shape[0], aal_data.shape[1] / pet_scan.shape[1], aal_data.shape[2] / pet_scan.shape[2]))

In [ ]:
plt.imshow(pet_scan_resized[:, :, 40])

# Extraction
---

## Available regions

In [ ]:
aal_labels

In [ ]:
a_file = open("labels.pkl", "wb")
pickle.dump(aal_labels, a_file)
a_file.close()

## Mask

In [ ]:
mask = aal_data != aal_labels['Hippocampus_L']

In [ ]:
plt.imshow(mask[:, :, 35])

## Extraction

In [ ]:
pet_scan_masked = copy.copy(pet_scan_resized)
pet_scan_masked[mask] = 0

In [ ]:
plt.imshow(pet_scan_masked[:, :, 30])